In [43]:
import csv
import pandas as pd


pd.set_option('display.max_rows', 500)

In [67]:
!head result/positive.tsv

weight	text
633	как варить 'борщ'(recipe/Recipe:borsch)
482	алиса как варить 'борщ'(recipe/Recipe:borsch)
400	как приготовить 'борщ'(recipe/Recipe:borsch)
378	как приготовить 'блины'(recipe/Recipe:blini)
313	как приготовить 'яичницу'(recipe/Recipe:fried_egg)
246	рецепт 'блинов'(recipe/Recipe:blini)
234	как варить 'яйца'(recipe_wildcard)
212	как варить 'гречку'(recipe_wildcard)
203	как варить 'пельмени'(recipe_wildcard)


In [51]:
data = []
with open('result/positive.tsv') as csvfile:
        reader = csv.reader(csvfile, delimiter='\t')
        for i, (weight, text) in enumerate(reader):
            if text != 'text':
                data.append({
                    'weight': int(weight),
                    'text': text
                })
                


In [65]:
import re

TEXT_RE = re.compile(r"'([a-zA-Zа-яА-Я ]+)'\(([a-zA-Z_/:]+)\)", re.UNICODE)

aliases = {}

def extract_recipe_name(text):
    match = TEXT_RE.search(text)
    if match is None:
        raise ValueError(f'failed to parse {text}')
    tag = match.group(2)
    
    if tag == 'recipe_wildcard':
        return match.group(1)
    else:
        if tag not in aliases:
            aliases[tag] = match.group(1)
        return tag
    
for row in data:
    row['recipe'] = extract_recipe_name(row['text'])
    


In [66]:
import collections

pd.set_option('display.max_rows', 500)

recipe_top = collections.defaultdict(lambda: 0)

for row in data:
    recipe_top[row['recipe']] += row['weight']
    
recipe_top_df = pd.DataFrame([
    {'recipe': aliases.get(recipe, recipe), 'weight': weight}
    for recipe, weight in recipe_top.items()
])
recipe_top_df.sort_values(by=['weight'], ascending=False, inplace=True)
with pd.option_context('display.max_rows', 100):
    display(recipe_top_df)
recipe_top_df.to_csv('popular_recipes.tsv', sep='\t')

,recipe,weight
0,борщ,3364
1,блины,2439
9,шарлотку,1325
2,яичницу,1045
5,пельмени,913
...,...,...
12926,пина,1
12925,печенья ушки,1
12924,печенья с шоколадом,1
12923,печенья с маргарином,1


In [ ]:
known_recipes = [
    'recipe/Recipe:borsch',
    'charlotte_apple_pie',
    'sirniki',
    'pizza',
    'casserole_generic',
    'casserole_tvorog',
    'russian_pancakes',
    'mannik',
    'blini',
    'plov',
    'okroshka',
    'zucchini_spread',
    'mimosa_salad',
    'kharcho',
    'lecho',
]